# 파이어베이스 (파이어스토어) 연결

https://firebase.google.com/docs/firestore?authuser=0


In [1]:
# cloud firestore 인스턴스 초기화(자체 서버에서 초기화)

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use the application default credentials - GCP에서 초기화
cred = credentials.Certificate('../firebase-apikey.json')
firebase_admin.initialize_app(cred)
db = firestore.client()


# 사용법


## 1. 참조
> users_ref = db.collection(u'users')

> docs_ref = db.collection(u'users').document(u'doc')

참조는 컬랙션 - 다큐먼트 - 컬랙션 - 다큐먼트 의 순서로만 가능하다

## 2. 데이터 쓰기

데이터 쓰기는 set() 메소드를 사용한다

> data = {
    u'name' : u'Los Angeles',
    u'state' : u'CA'
}

> db.collection(u'cities').document(u'LA').set(data)

유의미한 문서ID를 두고싶지 않을 때에는 add() 메소드를 사용한다. add() = .doc().set()

## 3. 데이터 업데이트 - 전체를 덮어쓰지 않고 일부 필드를 업데이트

업데이트는 update() 메소드를 사용한다<br>
배열 요소 업데이트는 arrayUnion()과 arrayRemove()를 사용한다.
> city_ref.update({u'regions' : firestore.ArrayUnion(\[u'greater_virginia'\])})

## 4. 데이터 가져오기

데이터 가져오기는 get() 메소드를 사용한다<br>
where() 메소드를 통해 컬랙션에서 특정 문서들만 뽑을 수 있다
> docs = db.collection(u'cities').where(u'capital', u'==', True).stream()

# 예시코드 (실행시 주의)

In [2]:
# 예시코드 - 콜랙션 내 모든 문서 불러오기

users_ref = db.collection(u'youtuber_keywords')
docs = users_ref.stream()

for doc in docs:
    print(u'{} => {}'.format(doc.id,doc.to_dict()))

youtube_keywords => {'youtube_title': '기리TV', 'keywords': ['플스5'], 'namuwiki_title': '기리TV'}


In [14]:
# 예시코드 - json 파일 읽고 db에 넣기

import json

with open('./crawling_result_text/youtuber_keywords.json', 'rt', encoding='utf-8') as f:
    json_data = json.load(f)

# print(json.dumps(json_data,ensure_ascii=False))


doc_ref = db.collection(u'youtuber').document(u'youtube_keywords')

for youtuber in json_data['youtubers']:
    doc_ref.set({
        u'youtube_title':youtuber['youtube_title'],
        u'namuwiki_title':youtuber['namuwiki_title'],
        u'keywords':youtuber['keywords']
    })


# 실제 코드(실행시 주의)

In [2]:
from googleapiclient.discovery import build
import os
import pandas as pd
import json


apiKey_dirt = 'APIkey.json'

#api 키 로드
with open( apiKey_dirt) as json_file:
    json_data = json.load(json_file)
    youtubeApiKey = json_data["APIkey"]

    
#채널 목록 로드
channel_list = pd.read_csv('channels.csv', header=0, sep=',')
channel_list.head(3)

,channel_ID,name_namuwiki
0,UChQ-VMvdGrYZxviQVMTJOHg,도티
1,UCbFzvzDu17eDZ3RIeaLRswQ,감스트
2,UCg7rkxrTnIhiHEpXY1ec9NA,잠뜰


In [13]:
# 채널 정보 저장하기

import re

youtube = build('youtube','v3',developerKey=youtubeApiKey)

for i in range(len(channel_list['channel_ID'])):
    #db 구조
    channel_data = {
        'youtube_title' : '',
        'namuwiki_title' : '',
        'channel_id' : '',
        'channel_desc' : '',
        'category' : [],
        'keywords' : []
    }
    # 채널 id, 나무위키 제목 저장
    channel_data['channel_id'] = channel_list['channel_ID'][i]
    channel_data['namuwiki_title'] = channel_list['name_namuwiki'][i]
    print("channel id : {} . namuwiki : {}".format(channel_data['channel_id'], channel_data['namuwiki_title']))

    # YOUTUBE API 연결
    channels_api = youtube.channels().list(part='brandingSettings, topicDetails', id=channel_data['channel_id']).execute()
    print("api ok")

    # 유튜브 제목, 채널 소개 저장
    channel_data['youtube_title'] = channels_api['items'][0]['brandingSettings']['channel']['title']
    try:
        channel_data['channel_desc'] = channels_api['items'][0]['brandingSettings']['channel']['description']
    except:
        print('no description')
    print("youtube title : {}. \ndesc : {}".format(channel_data['youtube_title'], channel_data['channel_desc']))

    # 채널 키워드 저장
    try:
        keyword_list = channels_api['items'][0]['brandingSettings']['channel']['keywords'].split()
        for keyword in keyword_list:
            channel_data['keywords'].append(keyword)
        print("keywords {}".format(channel_data['keywords']))
    except :
        print("no keywords")

    # 카테고리 저장
    category_list =  channels_api['items'][0]['topicDetails']['topicCategories']
    for category in category_list:
        clean_category = re.sub(pattern='https://en.wikipedia.org/wiki/', repl='', string=category)
        channel_data['category'].append(clean_category)
    print('category : {}'.format(channel_data['category']))


    # firestore 저장
    # doc_title = re.sub(pattern=' ', repl='', string=channel_data['youtube_title'])
    doc_title = channel_data['youtube_title']
    youtuber_ref = db.collection(u'Youtuber').document(doc_title)
    youtuber_ref.set(channel_data)
    print(doc_title + ' firestore ok')




channel id : UChQ-VMvdGrYZxviQVMTJOHg . namuwiki : 도티
api ok
youtube title : 도티 TV. 
desc : *With SANDBOX Network*

▶인스타그램(@ddotty.heesun) : https://www.instagram.com/ddotty.heesun

채널을 구독 하시면 매일매일 업로드 되는 재밌는 영상들을 쉽고 편하게 확인 하실 수 있습니다!!
시청자분들과 함께 호흡하고 소통하는 채널이 될 수 있도록 성실히 노력하겠습니다 :D

#마인크래프트 #모바일게임 #종합게임 #Vlog

▷ 재생목록을 통해 영상을 시청하시면 시리즈 컨텐츠를 순서대로 편하게 보실 수 있습니다.

※각종 문의: ddotty@sandboxnetwork.net
keywords ['마인크래프트', '도티', '마크', '로블록스', '모바일게임', '클래시로얄']
category : ['Video_game_culture', 'Role-playing_video_game', 'Action_game']
도티 TV firestore ok
channel id : UCbFzvzDu17eDZ3RIeaLRswQ . namuwiki : 감스트
api ok
youtube title : 감스트GAMST. 
desc : 재미와 감동을 동시에 느낄 수 있는
감스트의 공식 유튜브 채널 (감튜브) 입니다!

감스트와 같이 찢으실 분들은
'구독' 눌러주시고 많이 시청하러 오세요 :-D

To have fun and be touched at the same time
The official YouTube channel of GAMST (GAMTUBE) *_*

If you want to have fun with GAMST
Please visit and subscribe to the channel for more videos
keywords ['감스트', 'gamst', '김인직', 'kiminjik', '먹방', 'mukbang', 'meokbang',

In [21]:
# 인기 동영상 (최대)50개 저장하기
# 영상 설명과 태그, 댓글은 다시 수집해야됨

import re

youtube = build('youtube','v3',developerKey=youtubeApiKey)

#db에서 목록 가져오기
youtuber_ref = db.collection(u'Youtuber')
docs = youtuber_ref.stream()

i = 0
max = 20

for doc in docs:
    if i == 0 :
        i = i+1
        continue
    elif i > max :
        break
    
    
    print("current number : {}".format(i))
    i = i+1

    doc_id = doc.id
    print("current youtuber : {}".format(doc_id))

    data = {
        'video_title' : '',
        'video_id' : '',
        'video_desc' : '',
        'tags' : []

    }
    doc_dic = doc.to_dict()
    channel_id = doc_dic['channel_id']

    search = youtube.search().list(part='id, snippet', channelId=channel_id, type='video', order='viewCount', maxResults='50').execute()
    print("YOUTUBE api ok")

    for video in search['items']:
        video_ref = db.collection(u'Youtuber').document(doc_id).collection(u'Popular_videos')

        data['video_id'] = video['id']['videoId']
        data['video_title'] = video['snippet']['title']
        try:
            data['video_desc'] = video['snippet']['description']
        except:
            print("no desc")
        print("data {}".format(data) )

        video_ref.add(data)
        print(channel_id + "firestore ok")
    
    print("youtuber {} OK===============".format(doc_id))







current number : 1
current youtuber : GreatMoonAroma 대월향
YOUTUBE api ok
data {'video_title': '🍞외국인 친구에게 아바타를 만들어 줬더니ㅋㅋㅋ 😍 【VR챗 레전드 하이라이트 모음집】 #20', 'video_id': 'dDr0kCE2s34', 'video_desc': 'VR챗 #VRchat #VRchat하이라이트 디스코드 채널 : discord.gg/McdSA6p 트위치 생방 : https://www.twitch.tv/greatmoonaroma 외국인 친구에게 아바타를 만들어 ...', 'tags': []}
UCWL_9MaaMy8ObTDPmVHemjAfirestore ok
data {'video_title': '🥴️ This Is my voice 1 day in Soviet Russia 🍺【VRChat funny Highlights】 #49', 'video_id': 'iz03bk3_JEA', 'video_desc': 'meme #VRchat #Russia Discord : https://discord.gg/jjRNy6x twitch : https://www.twitch.tv/greatmoonaroma 🥴️ This Is my voice 1 day in Soviet Russia 【VRChat ...', 'tags': []}
UCWL_9MaaMy8ObTDPmVHemjAfirestore ok
data {'video_title': '🕴🕴🕴🕴 COFFIN DANCE MEME 💀 【VRChat funny Highlights】 #46', 'video_id': 'x7cLot3a7d8', 'video_desc': 'meme #VRchat #coffinmeme Discord : https://discord.gg/jjRNy6x twitch : https://www.twitch.tv/greatmoonaroma COFFIN DANCE MEME 【VRChat funny ...', 'tags': []}
UCWL_9Ma